In [ ]:
import sys
import os

import numpy as np
import scipy.optimize as opt
import pandas as pd
from matplotlib import pyplot as plt, animation
from matplotlib.lines import Line2D
from matplotlib.patches import Ellipse
import proplot as plot
import seaborn as sns
from tqdm import tqdm

sys.path.append('/Users/46h/Research/code/accphys/')
from tools import plotting as myplt
from tools import animation as myanim
from tools import utils
from tools.utils import play
from tools import beam_analysis as ba

sys.path.append('../')
from data_analysis import to_vec, to_mat, reconstruct

In [ ]:
plot.rc['figure.facecolor'] = 'white'
plot.rc['savefig.dpi'] = 'figure'
plot.rc['animation.html'] = 'jshtml'
plot.rc['grid.alpha'] = 0.04
plot.rc['axes.grid'] = False

save_figures = True

In [ ]:
def save(figname):
    if save_figures:
        filename = os.path.join('_output/figures', figname + '.png')
        plt.savefig(filename, facecolor='white', dpi=250)

# 4D Measurement in RTBT
> This notebook reconstructs the beam covariance matrix at the entrance of the Ring to Target Beam Transport (RTBT) section of the Spallation Neutron Source (SNS).

<img src="_input/rtbt.png" width=800>

## Method summary

The goal is to reconstruct the transverse beam covariance matrix at position $s = s_0$:

$$
\Sigma_{0} = \begin{bmatrix}
    \langle{x^2}\rangle & \langle{xx'}\rangle & \langle{xy}\rangle & \langle{xy'}\rangle \\
    \langle{xx'}\rangle & \langle{{x'}^2}\rangle & \langle{yx'}\rangle & \langle{x'y'}\rangle \\
    \langle{xy}\rangle & \langle{yx'}\rangle & \langle{y^2}\rangle & \langle{yy'}\rangle \\
    \langle{xy'}\rangle & \langle{x'y'}\rangle & \langle{yy'}\rangle & \langle{{y'}^2}\rangle
\end{bmatrix}.
$$

To do this, a set of $n$ wire-scanners can be placed at positions $\{s_i\} > s_0$ with $i = 1, ..., n$. A single measurement from wire-scanner $i$ will produce the real-space moments of the beam at $s_i$: $\langle{x^2}\rangle_{i}$, $\langle{y^2}\rangle_{i}$, and $\langle{xy}\rangle_{i}$. Without space charge, the transfer matrix $M_{s_0 \rightarrow s_i} = M_i$ is known. The moments at $s_0$ are then directly related to those at $s_i$ by

$$\Sigma_i = M_i \Sigma_{0} {M_i}^T.$$ This gives <br>

$$
\begin{align}
    \langle{x^2}\rangle_i &= 
        m_{11}^2\langle{x^2}\rangle_{0} 
      + m_{12}^2\langle{x'^2}\rangle_{0} 
      + 2m_{11}m_{22}\langle{xx'}\rangle_{0} ,\\
    \langle{y^2}\rangle_i &= 
        m_{33}^2\langle{y^2}\rangle_{0} 
      + m_{34}^2\langle{y'^2}\rangle_{0} 
      + 2m_{33}m_{34}\langle{yy'}\rangle_{0} ,\\
    \langle{xy}\rangle_i &= 
        m_{11}m_{33}\langle{xy}\rangle_{0} 
      + m_{12}m_{33}\langle{yx'}\rangle_{0} 
      + m_{11}m_{34}\langle{xy'}\rangle_{0} 
      + m_{12}m_{34}\langle{x'y'}\rangle_{0} ,
\end{align}
$$

where $m_{lm}$ are the elements of the transfer matrix. Taking 3 measurements with different optics settings between $s_0$ and $s_i$ (and therefore different transfer matrices) gives the 10 equations necessary to solve for $\Sigma_0$; however, real measurements will be noisy, so it is better to take more measurements if possible. Given $N$ measurements, we can form a $3N \times 1$ observation array $b$ from the measured moments and a $3N \times 10$ coefficient array $A$ from the transfer matrix such that

$$\begin{align} \mathbf{A \sigma}_0 = \mathbf{b},\end{align}$$ 

where $\mathbf{\sigma}_0$ is a $10 \times 1$ vector of the moments at $s_0$. There are 5 wire-scanners in the RTBT which operate simultaneously, so if all these are used the coefficient array will be $15N \times 10$. We then choose $\mathbf{\sigma}_0$ such that $|\mathbf{A\sigma}_0 - \mathbf{b}|^2$ is minimized. 

## RTBT lattice functions 

In [ ]:
twiss = pd.read_csv('_output/data/twiss.dat')
ws_positions = np.loadtxt('_output/data/ws_positions.dat')

In [ ]:
fig, axes = plot.subplots(nrows=3, figsize=(6, 5), spany=False)
for scan_index in range(1, 2):
    twiss_df = pd.read_csv('_output/data/twiss_{}.dat'.format(scan_index))
    twiss_df[['s','bx','by']].plot('s', ax=axes[0], legend=False)
    twiss_df[['s','ax','ay']].plot('s', ax=axes[1], legend=False)
    twiss_df[['s','nux','nuy']].plot('s', ax=axes[2], legend=False)
    axes.format(cycle='colorblind')
axes.format(xlabel='s [m]', toplabels='RTBT lattice functions (default)', grid=False)
for ax, ylabel in zip(axes, [r'$\beta$ [m]', r'$\alpha$ [rad]', 'Frac. phase / ($2\pi$)']):
    ax.format(ylabel=ylabel);
    for ws_position in ws_positions:
        ax.axvline(ws_position, color='grey', ls='--', lw=0.5, alpha=0.5, zorder=0)
axes[0].legend(labels=['x', 'y', 'wire-scanner'], ncols=1, loc=(1.02, 0));
save('twiss_scan')

In [ ]:
fig, ax = plot.subplots(figsize=(5, 2))
twiss[['s','bx','by']].plot('s', ax=ax, legend=False)
ax.format(xlabel='Position [m]', ylabel=r'$\beta$ [m]', toplabels='RTBT lattice functions')
for ws_position in ws_positions:
    ax.axvline(ws_position, color='grey', ls='--', lw=0.5, zorder=0)
ax.format(xlim=(0, twiss['s'].max()))
ax.legend(labels=[r'$\beta_x$', r'$\beta_y$', 'wire-scanner'], ncols=1, 
          fontsize=8, handlelength=1.0, frameon=False);
plt.savefig('_output/figures/beta.png', facecolor='white', dpi=250)

## Phase scan

In [ ]:
ws_ids = ['ws02', 'ws20', 'ws21', 'ws23', 'ws24']
active_ws_ids = ws_ids[:]

In [ ]:
def load(filename, ws_name):
    path = '_output/data/{}/{}'.format(ws_name, filename)
    return np.load(path)

phases_dict = {ws: load('phases.npy', ws) for ws in ws_ids}
moments_dict = {ws: 1e6 * load('moments.npy', ws) for ws in ws_ids}
transfer_mats_dict = {ws: load('transfer_mats.npy', ws) for ws in ws_ids}
env_params_dict = {ws: 1e3 * load('env_params.npy', ws) for ws in ws_ids}

phases = np.vstack([phases_dict[ws] for ws in active_ws_ids])
moments = np.vstack([moments_dict[ws] for ws in active_ws_ids]) # measured 
transfer_mats = np.vstack([transfer_mats_dict[ws] for ws in active_ws_ids])
env_params = np.vstack([env_params_dict[ws] for ws in active_ws_ids])
    
Sigma0 = np.loadtxt('_output/data/Sigma0.dat')
Sigma0 *= 1e6 # convert to [mm mrad]
X0 = np.loadtxt('_output/data/X0.dat')
X0 *= 1e3 # convert to [mm mrad]

Observe the beam at the wire-scanners. Any greyed-out wire-scanners are not used in the reconstruction.

In [ ]:
fig, axes = plot.subplots(nrows=2, ncols=5, figsize=(11, 3.75), spany=False)
plt_kws = dict(marker='.')
for ax, ws_name in zip(axes[0, :], ws_ids):
    ax.plot(phases_dict[ws_name] % 1, **plt_kws)
    ax.set_title(ws_name)
for ax, ws_name in zip(axes[1, :], ws_ids):
    ax.plot(moments_dict[ws_name][:, 0], **plt_kws)
    ax.plot(moments_dict[ws_name][:, 1], **plt_kws)
    ax.plot(moments_dict[ws_name][:, 2], **plt_kws)
axes[0, 0].legend(labels=['x', 'y'], ncols=3);
axes[1, 0].format(ylabel='[mm$^2$]')
axes[1, 0].legend(labels=[r'$\langle{x^2}\rangle$', r'$\langle{y^2}\rangle$', r'$\langle{xy}\rangle$'], fontsize='small');
axes[0, 0].format(ylabel='Frac. phase / ($2\pi$)', xlabel='Scan index')
save('ws_phase_adv')

In [ ]:
# fig, axes = plot.subplots(nrows=2, ncols=5, figsize=(8, 3.5), spany=False)
# for ax, ws_name in zip(axes[0, :], ws_ids):
#     ax.plot(phases_dict[ws_name] % 1, **plt_kws)
#     ax.set_title(ws_name, fontsize='large')
# for ax, ws_name in zip(axes[1, :], ws_ids):
#     ax.plot(moments_dict[ws_name][:, 0], **plt_kws)
#     ax.plot(moments_dict[ws_name][:, 1], **plt_kws)
#     ax.plot(moments_dict[ws_name][:, 2], **plt_kws)
# axes[0, 0].legend(labels=[r'$\nu_x$', r'$\nu_y$'], ncols=3);
# axes[1, 0].format(ylabel='[mm$^2$]')
# axes[1, 0].legend(labels=[r'$\langle{x^2}\rangle$', r'$\langle{y^2}\rangle$', r'$\langle{xy}\rangle$'], fontsize='small', ncols=2);
# axes[0, 0].format(ylabel='Frac. phase / ($2\pi$)', xlabel='Scan index', xlabel_kw={'size':'large'}, ylabel_kw={'size':'large'})
# plt.savefig('_output/figures/ws_phase_adv.png', facecolor='white', dpi=350)

To do: 
* Add column below showing phase vs scan index.
* Add diagonal line at angle of diagonal wire.

In [ ]:
# dims = ('x', 'y')

In [ ]:
# str_to_int = {'x':0, 'xp':1, 'y':2, 'yp':3}
# i, j = [str_to_int[dim] for dim in dims]

# ell_coords_list = [[ba.get_ellipse_coords(env_params) for env_params in env_params_dict[ws]] 
#                    for ws in ws_ids]
# ell_coords_list = np.array(ell_coords_list)

# pad = 0.25
# limits_list = np.array([(1 + pad) * myplt.max_u_up_global(coords) 
#                         for coords in ell_coords_list])
# umax, upmax = np.max(limits_list, axis=0)
# limits = 2 * [(-umax, umax), (-upmax, upmax)]
# labels = ['x [mm]', "x' [mrad]", 'y [mm]', "y' [mrad]"]

# fig, ax_list = plt.subplots(ncols=6, figsize=(13, 1.75), sharex=True, sharey=True)
# axes, text_ax = ax_list[:-1], ax_list[-1]
# text_ax.grid(False)
# myplt.despine([text_ax], 'all')
# myplt.despine(axes)
# axes[0].set_xlim(limits[i])
# axes[0].set_ylim(limits[j])
# axes[0].set_yticks(axes[0].get_xticks())
# axes[0].set_xlabel(labels[i])
# axes[0].set_ylabel(labels[j])
# for ax, ws_name in zip(axes, ws_ids):
#     ax.set_title(ws_name, color='grey' if ws_name not in active_ws_ids else 'k')
#     ax.grid(False)
#     ax.axvline(0, lw=0.2, c='k', alpha=0.2, zorder=99)
#     ax.axhline(0, lw=0.2, c='k', alpha=0.2, zorder=99)
# plt.close()

# def update(t):
#     for ax, coords in zip(axes, ell_coords_list):
#         for patch in ax.patches:
#             patch.remove()
#         ax.fill(coords[t, :, i], coords[t, :, j], fc='lightsteelblue', ec='k', zorder=10)
#     for ws, ax in zip(ws_ids, axes):
#         nux, nuy = phases_dict[ws][t]
#         for text in ax.texts:
#             text.set_visible(False)
#         ax.annotate(r'$\nu_x = {:.2f}$'.format(nux), xy=(0.75, 0.85), xycoords='axes fraction', zorder=10)
#         ax.annotate(r'$\nu_y = {:.2f}$'.format(nuy), xy=(0.75, 0.75), xycoords='axes fraction', zorder=10)
#     for text in text_ax.texts:
#         text.set_visible(False)
#     text_ax.annotate('Scan index = {}'.format(t), xy=(0.25, 0.5), xycoords='axes fraction', horizontalalignment='center')

# nframes = ell_coords_list[0].shape[0]
# fps = 1
# anim = animation.FuncAnimation(fig, update, frames=nframes, interval=1000/fps)
# if save_figures:
#      anim.save('_output/figures/ws_envelope.mp4', dpi=300)
# anim

In [ ]:
# for ws in ws_ids:
#     print(ws)
#     anim = myanim.corner_env(env_params_dict[ws], figsize=(4, 4), text_fmt='Scan index = {}')
#     play(anim)

In [ ]:
# myplt.corner(X0, moments=True, text='Initial bunch');
# save('initial_dist')

## Reconstruction

In [ ]:
Sigma = reconstruct(transfer_mats, moments, verbose=2)
print()
utils.show(Sigma, 'Sigma')
print()
utils.show(Sigma0, 'Sigma0')
print()
utils.show(Sigma - Sigma0, 'Sigma - Sigma0')

axes = myplt.rms_ellipses(Sigma0, color='lightsteelblue', fill=True, lw=0);
axes = myplt.rms_ellipses(Sigma, axes=axes, color='red8', lw=1)
axes[1, 1].legend(labels=['True', 'Reconstructed'], loc=(0, 1.1))
save('projection_default')

## Visualization using lines

In [ ]:
ax0, ay0, bx0, by0 = [-0.25897, 0.9749, 2.2991, 14.2583]
ex, ey = ba.apparent_emittances(Sigma0)

def V_2D(alpha, beta):
    return np.array([[beta, 0.0], [-alpha, 1.0]]) / np.sqrt(beta)

V = np.zeros((4, 4))
V[:2, :2] = V_2D(ax0, bx0)
V[2:, 2:] = V_2D(ay0, by0)
Vinv = np.linalg.inv(V)

def normalize(vec):
    return np.matmul(Vinv, vec)

scale = 2
xmax = scale * np.sqrt(ex)
ymax = scale * np.sqrt(ey)

In [ ]:
def get_line_points(M, sig_xx, sig_yy, dim, upmin=-10, upmax=10):
    xrms, yrms = np.sqrt(sig_xx), np.sqrt(sig_yy)
    hpts, vpts = [], []
    for up in [upmin, upmax]:
        if dim == 'x':
            vec1 = [xrms, up, 0, 0]
            i, j = 0, 1
        elif dim == 'y':
            vec1 = [0, 0, yrms, up]
            i, j = 2, 3
        vec0 = np.matmul(np.linalg.inv(M), vec1)
        vec0 = normalize(vec0)
        hpts.append(vec0[i])
        vpts.append(vec0[j])
    return hpts, vpts

def plot_lines(ax, transfer_mats, moments, dim='x', **kws):  
    for transfer_mat, (sig_xx, sig_yy, sig_xy) in zip(transfer_mats, moments):
        hpts, vpts = get_line_points(transfer_mat, sig_xx, sig_yy, dim)
        ax.plot(hpts, vpts, **kws)
    return ax

In [ ]:
fig, axes = plot.subplots(ncols=2, sharex=False, sharey=False)
axes.format(aspect=1)
axes[0].format(xlim=(-xmax, xmax), ylim=(-xmax, xmax), xlabel=r"$x_n$ [mm]", ylabel=r"$x'_n$ [mrad]")
axes[1].format(xlim=(-ymax, ymax), ylim=(-ymax, ymax), xlabel=r"$y_n$ [mm]", ylabel=r"$y'_n$ [mrad]")
myplt.ellipse(axes[0], np.sqrt(ex), np.sqrt(ex), lw=2, zorder=999)
myplt.ellipse(axes[1], np.sqrt(ey), np.sqrt(ey), lw=2, zorder=999)

active_ws_ids = ['ws20', 'ws21', 'ws23', 'ws24']
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for ws_id, color in zip(active_ws_ids, colors):
    kws = dict(lw=2, color=color)
    plot_lines(axes[0], transfer_mats_dict[ws_id], moments_dict[ws_id], 'x', **kws);
    plot_lines(axes[1], transfer_mats_dict[ws_id], moments_dict[ws_id], 'y', **kws);
save('lines')

In [ ]:
# fig, axes = plot.subplots(ncols=2, sharex=False, sharey=False)
# axes.format(aspect=1)
# axes[0].format(xlim=(-xmax, xmax), ylim=(-xmax, xmax), xlabel=r"$x_n$ [mm]", ylabel=r"$x'_n$ [mrad]")
# axes[1].format(xlim=(-ymax, ymax), ylim=(-ymax, ymax), xlabel=r"$y_n$ [mm]", ylabel=r"$y'_n$ [mrad]")
# myplt.ellipse(axes[0], np.sqrt(ex), np.sqrt(ex), lw=2, zorder=999)
# myplt.ellipse(axes[1], np.sqrt(ey), np.sqrt(ey), lw=2, zorder=999)
# plt.close()

# def update(i):
#     plot_lines(axes[0], transfer_mats[:i+1], moments[:i+1], dim='x', color='pink9')
#     plot_lines(axes[1], transfer_mats[:i+1], moments[:i+1], dim='y', color='pink9')
    
# animation.FuncAnimation(fig, update, frames=len(transfer_mats))

I thought maybe we could do something similar for the other projections, but it doesn't seem to work since the transfer matrix is completely uncoupled. For example, choosing a different $y'$ at the measurement location has no effect on the $x$ at the reconstruction location, so you just get a series of vertical lines in the $x$-$y'$ plane.

In [ ]:
def get_line_points_xyp(M, sig_xx, sig_yy, sig_xy, ypmin=-10, ypmax=10):
    xrms, yrms = np.sqrt(sig_xx), np.sqrt(sig_yy)
    r = sig_xy / (xrms * yrms)
    hpts, vpts = [], []
    for yp in [ypmin, ypmax]:
        vec1 = [xrms, 0, 0, yp]
        vec0 = np.matmul(np.linalg.inv(M), vec1)
        hpts.append(vec0[0])
        vpts.append(vec0[3])
    return hpts, vpts

def plot_lines_xyp(ax, transfer_mats, moments):  
    for M, (sig_xx, sig_yy, sig_xy) in zip(transfer_mats, moments):
        hpts, vpts = get_line_points_xyp(M, sig_xx, sig_yy, sig_xy)
        ax.plot(hpts, vpts, c='pink8')
    return ax
        
fig, ax = plot.subplots(figsize=(3, 3))
ax.format(xlim=(-xmax, xmax), ylim=(-ymax, ymax), xlabel=r"$x_n$ [mm]", ylabel=r"$y'_n$ [mrad]")
plot_lines_xyp(ax, transfer_mats, moments);
ax.format(xlim=(-30, 30), ylim=(-20, 20))